In [141]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3"

In [138]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.layers import CuDNNLSTM, Dense, Input, Dropout, Activation, Concatenate
from keras.optimizers import Adam

from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import multi_gpu_model
from tqdm import tqdm

In [84]:
demo = pd.read_csv('dataset/demo.csv')
demo.rename({
    'c0':'gender',
    'c1':'ocp'}, axis=1, inplace=True)

txn = pd.read_csv('dataset/txn.csv')
_txn = txn.copy()
txn.rename({
    'n3':'date_idx'
}, axis=1, inplace=True)
txn.drop('t0', axis=1, inplace=True)

train_set = pd.read_csv('dataset/train.csv')
test_set = pd.read_csv('dataset/test.csv')

In [3]:
id_ccno = txn[['id', 'old_cc_no']].drop_duplicates()
train_id_ccno = pd.merge(id_ccno, train_set, on='id')

In [4]:
txn_n = txn[['id', 'old_cc_no', 'date_idx', 'n4', 'n5', 'n6', 'n7']].copy()
txn_n['count'] = 1

In [5]:
LENGTH_SEQ = 365
LENGTH_SEQ = 53

In [6]:
txn_n['date_idx'] = txn_n['date_idx'] // 7 + 1 # WEEKLY
txn_n = txn_n.groupby(['id', 'old_cc_no', 'date_idx']).sum()

In [7]:
scaled_txn_n = txn_n

In [8]:
scaled_txn_n.head()

n4    n5     n6     n7  count
id old_cc_no date_idx                                  
1  7051      1         89000  5181  25825  11475     51
             2         25000  4286  20996   9450     42
             3         53500  4155  20589   9225     41
             4         28300  2562  12798   5625     25
             5         18000  3284  16822   7200     32

In [9]:
txn_scaler = StandardScaler()
scaled_txn_n[['n4', 'n5', 'n6', 'n7', 'count']] = txn_scaler.fit_transform(txn_n)

In [10]:
scaled_txn_n = scaled_txn_n.reset_index().set_index('old_cc_no')

In [11]:
txn[['old_cc_label', 'c5', 'c6', 'c7']].head()

,old_cc_label,c5,c6,c7
0,10,14,10,58
1,10,11,-10,53
2,10,12,-10,58
3,10,11,-10,80
4,10,12,11,58


In [14]:
txn.head()

,id,old_cc_label,c5,c6,c7,date_idx,n4,n5,n6,n7
old_cc_no,,,,,,,,,,
256451,11592,10,14,10,58,296,500,159,600,3801
256451,11592,10,11,-10,53,218,500,97,800,3801
256451,11592,10,12,-10,58,133,700,101,300,3801
256451,11592,10,11,-10,80,340,4000,95,0,3801
256451,11592,10,12,11,58,306,400,124,300,3801


In [152]:
def preprocess_credit_class(old_cc_no_ids):

    txn_joined = _txn.drop_duplicates(subset='old_cc_no').set_index('old_cc_no')
    txn_joined = txn_joined.loc[old_cc_no_ids]

    txn_joined = txn_joined.drop(['n3', 'n4', 'n5', 'n6', 'n7', 't0'], axis=1)
    txn_joined = txn_joined.reset_index()

    _x = txn_joined[['old_cc_no','old_cc_label', 'c5', 'c6', 'c7']]


#     df = pd.DataFrame(columns=['old_cc_no', 'onehot'])
    dfs = []
    for (_, old_cc_no, old_cc_label, c5, c6, c7) in tqdm(_x.itertuples()):
        old_cc_label = to_categorical(old_cc_label, 13)
        c5 = to_categorical(c5, 100)
        c6 = to_categorical(c6, 79)
        c7 = to_categorical(c7, 95)
        onehot = np.concatenate((old_cc_label, c5, c6, c7))
#         df = df.append({
#             'old_cc_no': old_cc_no,
#             'onehot': onehot
#         }, ignore_index=True)

        dfs.append(pd.DataFrame({
            'old_cc_no': old_cc_no,
            'onehot': onehot
        }))
    return pd.concat(dfs,ignore_index=True)

In [153]:
padding_value = txn_scaler.transform([[0, 0, 0, 0, 0]])
def create_seq_from_group(group):
    seq = np.ones((LENGTH_SEQ, 5)) * padding_value
    group = group.sort_values('date_idx')
    for (_, _, date_idx, n4, n5, n6, n7, count) in group.itertuples():
        seq[int(date_idx) - 1] = [n4, n5, n6, n7, count]
    return seq

def create_seq(txn):
    grouped = txn.groupby(['old_cc_no'])
    seqs = []
    for name, group in tqdm(grouped):
        seq = create_seq_from_group(group)
        seqs.append(seq)
    seqs = np.array(seqs)
    return seqs

def create_seq_dataframe(txn):
    grouped = txn.groupby(['old_cc_no'])
    output_df = pd.DataFrame(columns=['old_cc_no', 'seq'])
    dfs = []
    for old_cc_no, group in tqdm(grouped):
        seq = create_seq_from_group(group)
#         output_df = output_df.append({
#             'old_cc_no': old_cc_no,
#             'seq': seq
#         }, ignore_index=True, )
        dfs.append(pd.DataFrame({
            'old_cc_no': old_cc_no,
            'seq': seq
        }))
    
#     return output_df
    return pd.concat(dfs, ignore_index=True)

In [156]:
batch_size = 32
def create_data(ids, is_test=False):
    
    train_old_cc_nos = pd.merge(train_id_ccno, ids, on='id')['old_cc_no']

    train_txn = scaled_txn_n.loc[train_old_cc_nos]
    train_txn = train_txn.sort_values(['old_cc_no', 'date_idx'])

    seqs = create_seq(train_txn)
#     return seqs
    onehot = preprocess_credit_class(train_old_cc_nos.to_list())
    onehot = np.array([v for v in onehot['onehot'].to_numpy()])
    if is_test:
        return onehot, seqs
    else:
        labels = train_id_ccno.set_index('old_cc_no').loc[train_old_cc_nos].sort_values('old_cc_no')['label']
        labels = np.array([to_categorical(label, 13) if not np.isnan(label) else to_categorical(0, 13) for label in labels])
        return onehot, seqs, labels

In [3]:
seqs = np.load('data_saved/seq-weekly.npy')
labels = np.load('data_saved/label-weekly.npy')

In [134]:
ids = train_set['id']
onehot, seqs, labels = create_data(ids)
# train_old_cc_nos = create_data(ids)


  7%|▋         | 10700/150511 [00:14<03:03, 760.69it/s]


 14%|█▍        | 21313/150511 [00:28<02:49, 761.18it/s]


 21%|██        | 31925/150511 [00:42<02:35, 763.86it/s]


 28%|██▊       | 42507/150511 [00:56<02:21, 761.74it/s]


 35%|███▌      | 53072/150511 [01:10<02:08, 760.83it/s]


...

140847it [13:53, 86.20it/s]
140856it [13:54, 86.20it/s]
140865it [13:54, 86.23it/s]
140874it [13:54, 86.26it/s]
140883it [13:54, 86.29it/s]
140892it [13:54, 86.29it/s]
140901it [13:54, 86.26it/s]
140910it [13:54, 86.23it/s]
140919it [13:54, 86.27it/s]
140928it [13:54, 86.26it/s]
140937it [13:54, 85.58it/s]
140946it [13:55, 85.59it/s]
140955it [13:55, 85.76it/s]
140964it [13:55, 85.90it/s]
140973it [13:55, 86.01it/s]
140982it [13:55, 85.94it/s]
140991it [13:55, 85.64it/s]
141000it [13:55, 85.62it/s]
141009it [13:55, 85.03it/s]
141018it [13:55, 85.37it/s]
141027it [13:56, 85.64it/s]
141036it [13:56, 85.81it/s]
141045it [13:56, 85.36it/s]
141054it [13:56, 85.61it/s]
141063it [13:56, 85.79it/s]
141072it [13:56, 85.84it/s]
141081it [13:56, 85.95it/s]
141090it [13:56, 85.82it/s]
141099it [13:56, 85.67it/s]
141108it [13:56, 85.72it/s]
141117it [13:57, 85.85it/s]
141126it [13:57, 85.95it/s]
141135it [13:57, 86.00it/s]
141144it [13:57, 86.00it/s]
141153it [13:57, 86.04it/s]
141162it [13:57, 86.

143223it [14:21, 85.43it/s]
143232it [14:21, 85.44it/s]
143241it [14:21, 85.43it/s]
143250it [14:21, 85.49it/s]
143259it [14:22, 85.52it/s]
143268it [14:22, 85.55it/s]
143277it [14:22, 85.54it/s]
143286it [14:22, 85.43it/s]
143295it [14:22, 85.46it/s]
143304it [14:22, 85.48it/s]
143313it [14:22, 85.50it/s]
143322it [14:22, 85.34it/s]
143331it [14:22, 85.36it/s]
143340it [14:23, 85.35it/s]
143349it [14:23, 85.35it/s]
143358it [14:23, 85.39it/s]
143367it [14:23, 85.46it/s]
143376it [14:23, 85.52it/s]
143385it [14:23, 85.50it/s]
143394it [14:23, 85.51it/s]
143403it [14:23, 85.54it/s]
143412it [14:23, 85.52it/s]
143421it [14:23, 85.43it/s]
143430it [14:24, 85.46it/s]
143439it [14:24, 85.47it/s]
143448it [14:24, 85.38it/s]
143457it [14:24, 85.37it/s]
143466it [14:24, 85.09it/s]
143475it [14:24, 85.20it/s]
143484it [14:24, 85.27it/s]
143493it [14:24, 85.34it/s]
143502it [14:24, 85.37it/s]
143511it [14:25, 85.39it/s]
143520it [14:25, 85.45it/s]
143529it [14:25, 85.45it/s]
143538it [14:25, 85.

145599it [14:49, 84.31it/s]
145608it [14:49, 84.32it/s]
145617it [14:49, 84.26it/s]
145626it [14:50, 84.25it/s]
145635it [14:50, 84.28it/s]
145644it [14:50, 84.15it/s]
145653it [14:50, 84.20it/s]
145662it [14:50, 84.27it/s]
145671it [14:50, 84.20it/s]
145680it [14:50, 84.23it/s]
145689it [14:50, 83.54it/s]
145698it [14:50, 83.75it/s]
145707it [14:50, 83.93it/s]
145716it [14:51, 84.02it/s]
145725it [14:51, 84.10it/s]
145734it [14:51, 84.13it/s]
145743it [14:51, 84.10it/s]
145752it [14:51, 84.15it/s]
145761it [14:51, 84.11it/s]
145770it [14:51, 84.12it/s]
145779it [14:51, 84.09it/s]
145788it [14:51, 84.12it/s]
145797it [14:52, 84.16it/s]
145806it [14:52, 84.18it/s]
145815it [14:52, 84.22it/s]
145824it [14:52, 83.82it/s]
145833it [14:52, 83.91it/s]
145842it [14:52, 84.01it/s]
145851it [14:52, 84.12it/s]
145860it [14:52, 84.16it/s]
145869it [14:52, 84.12it/s]
145878it [14:53, 84.15it/s]
145887it [14:53, 84.19it/s]
145896it [14:53, 84.23it/s]
145905it [14:53, 84.12it/s]
145914it [14:53, 84.

147975it [15:18, 83.00it/s]
147984it [15:18, 83.04it/s]
147993it [15:18, 83.04it/s]
148002it [15:18, 83.09it/s]
148011it [15:18, 83.11it/s]
148020it [15:18, 83.07it/s]
148029it [15:18, 83.05it/s]
148038it [15:18, 82.99it/s]
148047it [15:18, 83.00it/s]
148056it [15:19, 82.95it/s]
148065it [15:19, 82.94it/s]
148074it [15:19, 82.88it/s]
148083it [15:19, 82.94it/s]
148092it [15:19, 82.97it/s]
148101it [15:19, 82.96it/s]
148110it [15:19, 83.00it/s]
148119it [15:19, 83.01it/s]
148128it [15:19, 83.00it/s]
148137it [15:20, 83.02it/s]
148146it [15:20, 83.03it/s]
148155it [15:20, 83.02it/s]
148164it [15:20, 82.95it/s]
148173it [15:20, 82.94it/s]
148182it [15:20, 82.94it/s]
148191it [15:20, 82.95it/s]
148200it [15:20, 82.92it/s]
148209it [15:20, 82.97it/s]
148218it [15:21, 82.98it/s]
148227it [15:21, 82.96it/s]
148236it [15:21, 82.96it/s]
148245it [15:21, 82.91it/s]
148254it [15:21, 82.94it/s]
148263it [15:21, 82.91it/s]
148272it [15:21, 82.87it/s]
148281it [15:21, 82.90it/s]
148290it [15:21, 82.

150351it [15:46, 81.84it/s]
150360it [15:47, 81.84it/s]
150369it [15:47, 81.82it/s]
150378it [15:47, 81.86it/s]
150387it [15:47, 81.82it/s]
150396it [15:47, 81.79it/s]
150405it [15:47, 81.77it/s]
150414it [15:47, 81.64it/s]
150423it [15:47, 81.70it/s]
150432it [15:47, 81.74it/s]
150441it [15:48, 81.79it/s]
150450it [15:48, 81.69it/s]
150459it [15:48, 81.70it/s]
150468it [15:48, 81.73it/s]
150477it [15:48, 81.75it/s]
150486it [15:48, 81.71it/s]
150495it [15:48, 81.76it/s]
150504it [15:48, 81.73it/s]
150511it [15:48, 158.61it/s]

In [135]:
onehot.shape, seqs.shape, labels.shape

((150511, 287), (150511, 53, 5), (150511, 13))

In [136]:
np.save('data_saved/class.npy', onehot)
np.save('data_saved/seq-weekly.npy', seqs)
np.save('data_saved/label-weekly.npy', labels)


In [118]:
def build_class_model(n_input):
    model = Sequential()
    
    model.add(Dense(64, activation='relu', input_shape=(n_input, )))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(16, activation='relu'))
#     model.add(Dense(13, activation='softmax'))
    
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [119]:
def build_txn_model():
    model = Sequential()
    model.add(CuDNNLSTM(32, return_sequences=True, input_shape=(LENGTH_SEQ, 5)))
#     model.add(CuDNNLSTM(32, return_sequences=True))
#     model.add(CuDNNLSTM(32, return_sequences=True))
    model.add(CuDNNLSTM(32))
#     model.add(Dense(13, activation='softmax'))
    model.summary()
    # model = multi_gpu_model(model, gpus=2)
#     model.compile(optimizer=Adam(),
#                   loss='categorical_crossentropy',
#                  metrics=['acc'])
    return model



In [120]:
class_model = build_class_model(287)
class_model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                18432     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_19 (Dense)             (None, 16)                2064      
Total params: 45,328
Trainable params: 45,328
Non-trainable params: 0
__________________________________________________

In [121]:
txn_model = build_txn_model()
txn_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 53, 32)            4992      
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 53, 32)            8448      
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 53, 32)            8448      
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 32)                8448      
Total params: 30,336
Trainable params: 30,336
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 53, 32)            4992      
____________________________

In [145]:
x = Concatenate()([class_model.output, txn_model.output])
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(13, activation='softmax')(x)
model = Model(inputs=[class_model.input, txn_model.input], output = x)
# model = multi_gpu_model(model, gpus=3)
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_16_input (InputLayer)     (None, 287)          0                                            
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 64)           18432       dense_16_input[0][0]             
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 64)           0           dense_16[0][0]                   
__________________________________________________________________________________________________
cu_dnnlstm_1_input (InputLayer) (None, 53, 5)        0                                            
____________________________________________________________________________________________

/home/porlolicon/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """


In [146]:
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
             metrics=['acc'])

In [147]:
class_weights = [1.09518711e+01, 3.07683547e+00, 3.58078232e-01, 2.61727260e-01,
       4.38833908e-01, 1.35073077e+03, 6.72563039e+00, 9.89546351e+00,
       3.99230769e+00, 1.03902367e+01, 4.49295078e-01, 4.26546559e+01,
       1.18329459e+00]

log_dir = 'weights/ensemble'
model.fit([onehot, seqs],
          labels,
          batch_size=128,
          shuffle=True,
          class_weight=class_weights,
          validation_split=0.3,
          callbacks=[
              TensorBoard(log_dir),
              ModelCheckpoint(log_dir+"/ep{epoch:03d}-val_loss:{val_loss}-val_acc:{val_acc}.hdf5", 
                             monitor='val_loss',
                              mode='min',
                              save_best_only=True,
                             )
          ],
          epochs=100)

Train on 105357 samples, validate on 45154 samples
Epoch 1/100
105357/105357 [==============================] - 43s 412us/step - loss: 1.7648 - acc: 0.3401 - val_loss: 1.6954 - val_acc: 0.3515
Epoch 2/100
105357/105357 [==============================] - 42s 403us/step - loss: 1.7356 - acc: 0.3508 - val_loss: 1.6582 - val_acc: 0.3666
Epoch 3/100
105357/105357 [==============================] - 43s 404us/step - loss: 1.7187 - acc: 0.3575 - val_loss: 1.6593 - val_acc: 0.3707
Epoch 4/100
105357/105357 [==============================] - 43s 406us/step - loss: 1.7125 - acc: 0.3595 - val_loss: 1.6504 - val_acc: 0.3696
Epoch 5/100
105357/105357 [==============================] - 42s 403us/step - loss: 1.7067 - acc: 0.3622 - val_loss: 1.6613 - val_acc: 0.3699
Epoch 6/100
105357/105357 [==============================] - 43s 404us/step - loss: 1.7036 - acc: 0.3638 - val_loss: 1.6406 - val_acc: 0.3769
Epoch 7/100
105357/105357 [==============================] - 42s 402us/step - loss: 1.6987 - acc:

Epoch 58/100
105357/105357 [==============================] - 42s 403us/step - loss: 1.5713 - acc: 0.4163 - val_loss: 1.6901 - val_acc: 0.3717
Epoch 59/100
105357/105357 [==============================] - 43s 404us/step - loss: 1.5697 - acc: 0.4189 - val_loss: 1.6980 - val_acc: 0.3708
Epoch 60/100
105357/105357 [==============================] - 42s 399us/step - loss: 1.5653 - acc: 0.4187 - val_loss: 1.6974 - val_acc: 0.3688
Epoch 61/100
105357/105357 [==============================] - 42s 401us/step - loss: 1.5631 - acc: 0.4203 - val_loss: 1.7090 - val_acc: 0.3669
Epoch 62/100
105357/105357 [==============================] - 42s 400us/step - loss: 1.5611 - acc: 0.4201 - val_loss: 1.7054 - val_acc: 0.3668
Epoch 63/100
105357/105357 [==============================] - 43s 405us/step - loss: 1.5588 - acc: 0.4211 - val_loss: 1.7088 - val_acc: 0.3675
Epoch 64/100
105357/105357 [==============================] - 42s 402us/step - loss: 1.5586 - acc: 0.4213 - val_loss: 1.7156 - val_acc: 0.3676

KeyboardInterrupt: 